# HyperTrader Strategy Backtest

This notebook backtests the HyperTrader 4-phase strategy using historical data.

**Strategy Overview:**
- **Advance Phase**: Both allocations long, tracking peaks
- **Retracement Phase**: Scaling positions during decline from peak
- **Decline Phase**: Long fully cashed, hedge fully short
- **Recovery Phase**: Systematic re-entry during recovery

---

## Setup

In [ ]:
# Package installation
%pip install pandas numpy matplotlib ccxt python-dotenv

# Imports
import sys
import numpy as np
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
from datetime import datetime
from decimal import Decimal, getcontext
from typing import Dict, Optional, Literal
import os
from dotenv import load_dotenv

# Set precision for financial calculations
getcontext().prec = 28

# Load environment variables
load_dotenv('../../.env')

## Configuration

In [ ]:
# Trading Configuration
symbol = "ETH/USDC:USDC"
timeframe = "1h"  # 1h for more granular testing
starting_date_backtest = "01 january 2024"
ending_date_backtest = "01 august 2024"
starting_date_dl = "01 december 2023"  # Extra data for warmup
ending_date_dl = "01 september 2024"

# HyperTrader Strategy Parameters
initial_margin = Decimal('1000')  # Starting capital in USDC
leverage = 1
unit_value_pct = Decimal('0.05')  # 5% of margin per unit
trade_fees = 0.045  # HyperLiquid fees in percent

# Risk Management
max_drawdown_threshold = Decimal('0.23')  # 23% max drawdown

print(f"Backtesting {symbol} from {starting_date_backtest} to {ending_date_backtest}")
print(f"Initial margin: ${initial_margin}")
print(f"Unit value: {float(unit_value_pct * 100)}% of margin")

## Data Download

In [ ]:
def download_data(symbol: str, timeframe: str, starting_date: str, ending_date: str) -> pd.DataFrame:
    """
    Download OHLCV data from Hyperliquid exchange using CCXT.
    """
    since = int(datetime.strptime(starting_date, "%d %B %Y").timestamp() * 1000)
    exchange = ccxt.hyperliquid({'enableRateLimit': True})
    
    print(f"Downloading {symbol} data...")
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since)
    
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    
    # Filter date range
    mask = (data['timestamp'] >= pd.to_datetime(starting_date)) & (data['timestamp'] <= pd.to_datetime(ending_date))
    data = data[mask]
    
    # Convert to numeric
    for col in ['open', 'high', 'low', 'close', 'volume']:
        data[col] = pd.to_numeric(data[col])
    
    print(f"Downloaded {len(data)} candles")
    return data

# Download historical data
ohlcv_data = download_data(symbol, timeframe, starting_date_dl, ending_date_dl)
print(f"Data shape: {ohlcv_data.shape}")
ohlcv_data.head()

## HyperTrader System State

In [ ]:
class SystemState:
    """HyperTrader system state for backtesting"""
    
    def __init__(self, entry_price: Decimal, initial_margin: Decimal, leverage: int = 1):
        # Basic Information
        self.entry_price = entry_price
        self.initial_margin = initial_margin
        self.leverage = leverage
        self.unit_value = initial_margin * unit_value_pct / leverage
        
        # Phase and Unit Tracking
        self.current_phase = "advance"
        self.current_unit = 0
        self.peak_unit = None
        self.valley_unit = None
        self.peak_price = entry_price
        self.valley_price = None
        
        # Allocations (50/50 split)
        allocation_amount = initial_margin / 2
        self.long_invested = allocation_amount
        self.long_cash = Decimal('0')
        self.hedge_long = allocation_amount
        self.hedge_short = Decimal('0')
        
        # Performance tracking
        self.total_fees = Decimal('0')
        self.trade_count = 0
        
    def calculate_current_unit(self, current_price: Decimal) -> int:
        """Calculate current unit position"""
        price_diff = current_price - self.entry_price
        return int(price_diff / self.unit_value)
    
    def get_total_portfolio_value(self) -> Decimal:
        """Calculate total portfolio value"""
        return self.long_invested + self.long_cash + self.hedge_long + self.hedge_short
    
    def is_reset_condition_met(self) -> bool:
        """Check if system reset conditions are met"""
        return self.hedge_short == 0 and self.long_cash == 0
    
    def is_choppy_trading_active(self) -> bool:
        """Check if choppy trading detection is active"""
        total_long = self.long_invested + self.long_cash
        long_partial = 0 < self.long_invested < total_long
        hedge_partial = self.hedge_long > 0 and self.hedge_short > 0
        return long_partial or hedge_partial

print("SystemState class defined")

## Trading Logic Implementation

In [ ]:
def determine_phase(state: SystemState) -> str:
    """Determine required trading phase based on current system state"""
    
    # Check allocation states
    long_fully_cash = (state.long_invested == 0 and state.long_cash > 0)
    hedge_fully_short = (state.hedge_long == 0 and state.hedge_short > 0)
    both_fully_long = (state.long_invested > 0 and state.hedge_long > 0 and state.hedge_short == 0)
    
    # Phase determination
    if long_fully_cash and hedge_fully_short:
        return 'decline'
    elif both_fully_long:
        return 'advance'
    elif state.peak_unit is not None and state.current_unit < state.peak_unit:
        return 'retracement'
    elif state.valley_unit is not None and state.current_unit > state.valley_unit:
        return 'recovery'
    else:
        return state.current_phase

def handle_advance_phase(state: SystemState) -> SystemState:
    """Handle advance phase: track peaks, maintain long positions"""
    # Update peak tracking
    if state.peak_unit is None or state.current_unit > state.peak_unit:
        state.peak_unit = state.current_unit
        state.peak_price = state.entry_price + (state.current_unit * state.unit_value)
    
    return state

def handle_retracement_phase(state: SystemState) -> SystemState:
    """Handle retracement phase: scale positions during decline"""
    if state.peak_unit is None:
        return state
    
    units_from_peak = state.current_unit - state.peak_unit
    is_choppy = state.is_choppy_trading_active()
    
    # Handle hedge allocation (immediate response)
    state = handle_hedge_retracement(state, units_from_peak)
    
    # Handle long allocation (with confirmation delay unless choppy)
    confirmation_needed = 1 if is_choppy else 2
    if abs(units_from_peak) >= confirmation_needed:
        state = handle_long_retracement(state, units_from_peak)
    
    return state

def handle_hedge_retracement(state: SystemState, units_from_peak: int) -> SystemState:
    """Handle hedge allocation scaling during retracement"""
    target_short_percentage = min(abs(units_from_peak) * 0.25, 1.0)
    target_long_percentage = 1.0 - target_short_percentage
    
    total_hedge = state.hedge_long + state.hedge_short
    target_long = total_hedge * Decimal(str(target_long_percentage))
    target_short = total_hedge * Decimal(str(target_short_percentage))
    
    # Execute position adjustment
    if abs(target_long - state.hedge_long) > Decimal('0.01'):
        state.hedge_long = target_long
        state.hedge_short = target_short
        state.trade_count += 1
        state.total_fees += total_hedge * Decimal(str(trade_fees / 100))
    
    return state

def handle_long_retracement(state: SystemState, units_from_peak: int) -> SystemState:
    """Handle long allocation scaling with confirmation delay"""
    confirmed_units = abs(units_from_peak) - 1
    target_invested_percentage = max(1.0 - (confirmed_units * 0.25), 0.0)
    
    total_long = state.long_invested + state.long_cash
    target_invested = total_long * Decimal(str(target_invested_percentage))
    target_cash = total_long - target_invested
    
    # Execute position adjustment
    if abs(target_invested - state.long_invested) > Decimal('0.01'):
        state.long_invested = target_invested
        state.long_cash = target_cash
        state.trade_count += 1
        state.total_fees += total_long * Decimal(str(trade_fees / 100))
    
    return state

def handle_decline_phase(state: SystemState) -> SystemState:
    """Handle decline phase: track valleys"""
    # Update valley tracking
    if state.valley_unit is None or state.current_unit < state.valley_unit:
        state.valley_unit = state.current_unit
        state.valley_price = state.entry_price + (state.current_unit * state.unit_value)
    
    return state

def handle_recovery_phase(state: SystemState) -> SystemState:
    """Handle recovery phase: systematic re-entry"""
    if state.valley_unit is None:
        return state
    
    units_from_valley = state.current_unit - state.valley_unit
    is_choppy = state.is_choppy_trading_active()
    
    # Handle hedge allocation (immediate response)
    state = handle_hedge_recovery(state, units_from_valley)
    
    # Handle long allocation (with confirmation delay unless choppy)
    confirmation_needed = 1 if is_choppy else 2
    if units_from_valley >= confirmation_needed:
        state = handle_long_recovery(state, units_from_valley)
    
    return state

def handle_hedge_recovery(state: SystemState, units_from_valley: int) -> SystemState:
    """Handle hedge allocation during recovery"""
    # Unwind shorts progressively
    total_hedge = state.hedge_long + state.hedge_short
    progress = min(units_from_valley * 0.25, 1.0)
    
    target_long = total_hedge * Decimal(str(progress))
    target_short = total_hedge - target_long
    
    if abs(target_long - state.hedge_long) > Decimal('0.01'):
        state.hedge_long = target_long
        state.hedge_short = target_short
        state.trade_count += 1
        state.total_fees += total_hedge * Decimal(str(trade_fees / 100))
    
    return state

def handle_long_recovery(state: SystemState, units_from_valley: int) -> SystemState:
    """Handle long allocation during recovery"""
    total_long = state.long_invested + state.long_cash
    progress = min((units_from_valley - 1) * 0.25, 1.0)
    
    target_invested = total_long * Decimal(str(progress))
    target_cash = total_long - target_invested
    
    if abs(target_invested - state.long_invested) > Decimal('0.01'):
        state.long_invested = target_invested
        state.long_cash = target_cash
        state.trade_count += 1
        state.total_fees += total_long * Decimal(str(trade_fees / 100))
    
    return state

def perform_system_reset(state: SystemState, current_price: Decimal) -> SystemState:
    """Perform system reset when conditions are met"""
    total_value = state.get_total_portfolio_value()
    allocation_amount = total_value / 2
    new_unit_value = total_value * unit_value_pct
    
    # Create new state
    new_state = SystemState(current_price, total_value, state.leverage)
    new_state.unit_value = new_unit_value
    new_state.long_invested = allocation_amount
    new_state.hedge_long = allocation_amount
    new_state.total_fees = state.total_fees
    new_state.trade_count = state.trade_count + 1
    
    return new_state

print("Trading logic functions defined")

## Main Trading Engine

In [ ]:
def on_unit_change(state: SystemState, current_price: Decimal) -> SystemState:
    """Main trading logic handler when unit changes"""
    
    # Update current unit
    new_unit = state.calculate_current_unit(current_price)
    state.current_unit = new_unit
    
    # Determine required phase
    required_phase = determine_phase(state)
    
    # Update phase if needed
    if required_phase != state.current_phase:
        state.current_phase = required_phase
    
    # Route to appropriate handler
    if state.current_phase == 'advance':
        state = handle_advance_phase(state)
    elif state.current_phase == 'retracement':
        state = handle_retracement_phase(state)
    elif state.current_phase == 'decline':
        state = handle_decline_phase(state)
    elif state.current_phase == 'recovery':
        state = handle_recovery_phase(state)
    
    # Check for system reset
    if state.is_reset_condition_met():
        state = perform_system_reset(state, current_price)
    
    return state

print("Main trading engine defined")

## Backtesting Engine

In [ ]:
def run_hypertrader_backtest(data: pd.DataFrame) -> tuple:
    """Run HyperTrader strategy backtest"""
    
    # Filter to backtest period
    backtest_data = data[
        (data['timestamp'] >= pd.to_datetime(starting_date_backtest)) & 
        (data['timestamp'] <= pd.to_datetime(ending_date_backtest))
    ].copy().reset_index(drop=True)
    
    if len(backtest_data) == 0:
        print("No data in backtest period!")
        return None, None
    
    # Initialize system state
    entry_price = Decimal(str(backtest_data.iloc[0]['close']))
    state = SystemState(entry_price, initial_margin, leverage)
    
    # Initialize tracking
    results = []
    last_unit = 0
    trades = []
    
    print(f"Starting backtest with entry price: ${entry_price}")
    print(f"Unit value: ${state.unit_value}")
    print(f"Processing {len(backtest_data)} candles...")
    
    # Process each candle
    for idx, row in backtest_data.iterrows():
        current_price = Decimal(str(row['close']))
        current_unit = state.calculate_current_unit(current_price)
        
        # Only act on unit changes
        if current_unit != last_unit:
            old_phase = state.current_phase
            old_portfolio_value = state.get_total_portfolio_value()
            
            # Execute trading logic
            state = on_unit_change(state, current_price)
            
            # Record trade if significant change occurred
            new_portfolio_value = state.get_total_portfolio_value()
            if abs(new_portfolio_value - old_portfolio_value) > Decimal('1'):
                trades.append({
                    'timestamp': row['timestamp'],
                    'price': float(current_price),
                    'unit_change': f"{last_unit} -> {current_unit}",
                    'phase_change': f"{old_phase} -> {state.current_phase}",
                    'portfolio_value': float(new_portfolio_value),
                    'total_fees': float(state.total_fees)
                })
            
            last_unit = current_unit
        
        # Calculate portfolio performance
        portfolio_value = state.get_total_portfolio_value()
        
        # Calculate unrealized PnL for positions
        # For simplicity, assume all positions scale with price proportionally
        price_ratio = current_price / entry_price
        
        # Long positions benefit from price increases
        long_value = (state.long_invested * price_ratio) + state.long_cash
        
        # Hedge positions: long benefits from increases, short benefits from decreases
        hedge_long_value = state.hedge_long * price_ratio
        hedge_short_value = state.hedge_short * (2 - price_ratio) if state.hedge_short > 0 else Decimal('0')
        
        total_unrealized_value = long_value + hedge_long_value + hedge_short_value
        
        # Record results
        results.append({
            'timestamp': row['timestamp'],
            'price': float(current_price),
            'current_unit': current_unit,
            'phase': state.current_phase,
            'portfolio_value': float(total_unrealized_value),
            'long_invested': float(state.long_invested),
            'long_cash': float(state.long_cash),
            'hedge_long': float(state.hedge_long),
            'hedge_short': float(state.hedge_short),
            'total_fees': float(state.total_fees),
            'trade_count': state.trade_count,
            'peak_unit': state.peak_unit,
            'valley_unit': state.valley_unit,
            'choppy_trading': state.is_choppy_trading_active()
        })
        
        # Progress indicator
        if idx % 500 == 0:
            print(f"Processed {idx}/{len(backtest_data)} candles...")
    
    results_df = pd.DataFrame(results)
    trades_df = pd.DataFrame(trades)
    
    print(f"Backtest completed! {len(trades)} trades executed.")
    
    return results_df, trades_df

print("Backtesting engine defined")

## Run Backtest

In [ ]:
# Run the backtest
results_df, trades_df = run_hypertrader_backtest(ohlcv_data)

if results_df is not None:
    print(f"\nBacktest Summary:")
    print(f"Total trades: {trades_df['portfolio_value'].count() if len(trades_df) > 0 else 0}")
    print(f"Starting portfolio: ${initial_margin}")
    print(f"Final portfolio: ${results_df['portfolio_value'].iloc[-1]:.2f}")
    print(f"Total return: {((results_df['portfolio_value'].iloc[-1] / float(initial_margin)) - 1) * 100:.2f}%")
    print(f"Total fees: ${results_df['total_fees'].iloc[-1]:.2f}")
    
    # Show first few results
    print("\nFirst 10 results:")
    display(results_df.head(10)[['timestamp', 'price', 'current_unit', 'phase', 'portfolio_value']])
    
    if len(trades_df) > 0:
        print("\nFirst 10 trades:")
        display(trades_df.head(10))

## Performance Analysis

In [ ]:
if results_df is not None and len(results_df) > 0:
    # Calculate performance metrics
    initial_value = float(initial_margin)
    final_value = results_df['portfolio_value'].iloc[-1]
    total_return_pct = ((final_value / initial_value) - 1) * 100
    
    # Calculate buy and hold performance
    initial_price = results_df['price'].iloc[0]
    final_price = results_df['price'].iloc[-1]
    hodl_return_pct = ((final_price / initial_price) - 1) * 100
    
    # Calculate max drawdown
    portfolio_values = results_df['portfolio_value']
    peak = portfolio_values.expanding().max()
    drawdown = (portfolio_values - peak) / peak
    max_drawdown_pct = drawdown.min() * 100
    
    # Trading statistics
    total_trades = results_df['trade_count'].iloc[-1] if 'trade_count' in results_df.columns else 0
    total_fees = results_df['total_fees'].iloc[-1]
    
    print(f"\n=== PERFORMANCE ANALYSIS ===")
    print(f"Period: {results_df['timestamp'].iloc[0]} to {results_df['timestamp'].iloc[-1]}")
    print(f"\n--- Returns ---")
    print(f"HyperTrader Strategy: {total_return_pct:.2f}%")
    print(f"Buy & Hold: {hodl_return_pct:.2f}%")
    print(f"Outperformance: {total_return_pct - hodl_return_pct:.2f}%")
    
    print(f"\n--- Risk Metrics ---")
    print(f"Maximum Drawdown: {max_drawdown_pct:.2f}%")
    print(f"Return/Drawdown Ratio: {total_return_pct / abs(max_drawdown_pct):.2f}" if max_drawdown_pct != 0 else "N/A")
    
    print(f"\n--- Trading Activity ---")
    print(f"Total Trades: {total_trades}")
    print(f"Total Fees: ${total_fees:.2f}")
    print(f"Fees as % of Initial Capital: {(total_fees / initial_value) * 100:.2f}%")
    
    # Phase distribution
    phase_distribution = results_df['phase'].value_counts()
    print(f"\n--- Phase Distribution ---")
    for phase, count in phase_distribution.items():
        print(f"{phase.capitalize()}: {count} periods ({count/len(results_df)*100:.1f}%)")

## Visualization

In [ ]:
if results_df is not None and len(results_df) > 0:
    # Create comprehensive visualization
    fig, axes = plt.subplots(4, 1, figsize=(15, 16))
    
    # Plot 1: Price and Units
    ax1 = axes[0]
    ax1_twin = ax1.twinx()
    
    ax1.plot(results_df['timestamp'], results_df['price'], color='blue', alpha=0.7, label='Price')
    ax1_twin.plot(results_df['timestamp'], results_df['current_unit'], color='red', alpha=0.7, label='Current Unit')
    
    ax1.set_ylabel('Price ($)', color='blue')
    ax1_twin.set_ylabel('Unit Position', color='red')
    ax1.set_title('Price Movement and Unit Position')
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')
    
    # Plot 2: Portfolio Performance
    ax2 = axes[1]
    
    # Calculate hodl performance
    hodl_values = (results_df['price'] / results_df['price'].iloc[0]) * float(initial_margin)
    
    ax2.plot(results_df['timestamp'], results_df['portfolio_value'], color='gold', linewidth=2, label='HyperTrader Strategy')
    ax2.plot(results_df['timestamp'], hodl_values, color='purple', alpha=0.7, label='Buy & Hold')
    ax2.axhline(y=float(initial_margin), color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
    
    ax2.set_ylabel('Portfolio Value ($)')
    ax2.set_title('Portfolio Performance Comparison')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Phase Distribution
    ax3 = axes[2]
    
    # Color code phases
    phase_colors = {'advance': 'green', 'retracement': 'orange', 'decline': 'red', 'recovery': 'blue'}
    for phase in results_df['phase'].unique():
        phase_data = results_df[results_df['phase'] == phase]
        ax3.scatter(phase_data['timestamp'], phase_data['portfolio_value'], 
                   c=phase_colors.get(phase, 'gray'), label=phase.capitalize(), alpha=0.6, s=1)
    
    ax3.set_ylabel('Portfolio Value ($)')
    ax3.set_title('Trading Phases Over Time')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Allocation Distribution
    ax4 = axes[3]
    
    ax4.fill_between(results_df['timestamp'], 0, results_df['long_invested'], 
                     alpha=0.7, label='Long Invested', color='green')
    ax4.fill_between(results_df['timestamp'], results_df['long_invested'], 
                     results_df['long_invested'] + results_df['long_cash'], 
                     alpha=0.7, label='Long Cash', color='lightgreen')
    ax4.fill_between(results_df['timestamp'], 
                     results_df['long_invested'] + results_df['long_cash'],
                     results_df['long_invested'] + results_df['long_cash'] + results_df['hedge_long'],
                     alpha=0.7, label='Hedge Long', color='blue')
    ax4.fill_between(results_df['timestamp'], 
                     results_df['long_invested'] + results_df['long_cash'] + results_df['hedge_long'],
                     results_df['long_invested'] + results_df['long_cash'] + results_df['hedge_long'] + results_df['hedge_short'],
                     alpha=0.7, label='Hedge Short', color='red')
    
    ax4.set_ylabel('Allocation ($)')
    ax4.set_xlabel('Time')
    ax4.set_title('Portfolio Allocation Over Time')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n=== FINAL RESULTS ===")
    print(f"Initial Capital: ${initial_margin}")
    print(f"Final Portfolio Value: ${results_df['portfolio_value'].iloc[-1]:.2f}")
    print(f"Total Return: {((results_df['portfolio_value'].iloc[-1] / float(initial_margin)) - 1) * 100:.2f}%")
    print(f"Buy & Hold Return: {hodl_return_pct:.2f}%")
    print(f"Strategy Outperformance: {total_return_pct - hodl_return_pct:.2f}%")
    print(f"Max Drawdown: {max_drawdown_pct:.2f}%")
    print(f"Total Trades: {total_trades}")
    print(f"Total Fees: ${total_fees:.2f}")

## Export Results

In [ ]:
# Save results to CSV for further analysis
if results_df is not None:
    results_df.to_csv('hypertrader_backtest_results.csv', index=False)
    print("Results saved to hypertrader_backtest_results.csv")
    
if trades_df is not None and len(trades_df) > 0:
    trades_df.to_csv('hypertrader_backtest_trades.csv', index=False)
    print("Trades saved to hypertrader_backtest_trades.csv")

print("\nBacktest completed successfully!")